In [33]:
import pandas as pd

In [34]:
common_characters_df = pd.read_csv('./data/common_hanzi.csv', index_col=0)
common_characters_df.head()

,汉字,常用
ID,,
1,一,1
2,乙,1
3,二,1
4,十,1
5,丁,1


In [35]:
common_characters_df = common_characters_df[common_characters_df['常用'] == 1]
common_characters = common_characters_df['汉字'].tolist()

# Generated Chinese Character Dataset

In [43]:
from PIL import ImageFont, ImageDraw, Image
from tqdm import tqdm
from pathlib2 import Path

In [44]:
font_tft_path = './data/font/STSONG.TTF'
text_size = 32 # also the image size
font = ImageFont.truetype(font_tft_path, text_size)
save_dir = Path('./data/common_hanzi')
if not save_dir.exists():
    save_dir.mkdir()

In [46]:
for character in tqdm(common_characters):
    text_width, text_height = font.getsize(character)
    xmin, ymin, xmax, ymax = font.getmask(character).getbbox()
    offsetx, offsety = font.getoffset(character)
    canvas = Image.new('L', (text_width, text_height), (255))
    draw = ImageDraw.Draw(canvas)
    white = 0
    draw.text((0, 0), character, font=font, fill=white)
    x_gap = (text_size - (xmax - xmin)) / 2
    y_gap = (text_size - (ymax - ymin)) / 2
    canvas = canvas.crop((xmin + offsetx - x_gap, ymin + offsety - y_gap, xmax + offsetx + x_gap, ymax + offsety + y_gap))
    # check the size of canvas
    assert canvas.size == (text_size, text_size)
    canvas.save(str(save_dir / '{}.png'.format(character)))

100%|██████████| 2500/2500 [00:01<00:00, 2135.40it/s]
